In [ ]:
import re #Biblioteca pra usar o re.compile e regular expressions


class Calc:
    def __init__(self, ip, mascara=None, prefixo=None):
        self.ip = ip
        self.mascara = mascara
        self.prefixo = prefixo

        self._set_broadcast()
        self._set_rede()

        print(f'O IP é: {self.ip}')
        print(f'A Mascara é: {self.mascara}')
        print(f'A rede é: {self.rede}')
        print(f'A Broadcast é: {self.broadcast}')
        print(f'O prefixo é: {self.prefixo}')
        print(f'Numero de IPS: {self._get_numero_ips()}')

    @property
    def rede(self): #Getter da _rede
        return self._rede

    @property
    def broadcast(self): #Getter da _broadcast 
        return self._broadcast


    @property
    def ip(self): #Getter do _ip
        return self._ip
    @property
    def mascara(self): #Getter do _mascara
        return self._mascara

    @property
    def prefixo(self): #Getter do _prefixo
        return self._prefixo

    @ip.setter
    def ip(self, valor):
        if not self._valida_ip(valor): #Usa regexp pra ver se veio estilo 000.000.000.000
            raise ValueError('ip errado') #Que são 4 blocos de entre 1 a 3 numeros de 0-9

        self._ip = valor
        self._ip_bin = self._ip_to_bin(valor) #Pega o valor do IP em binário

    @mascara.setter
    def mascara(self, valor):
        if not valor:
            return

        if not self._valida_ip(valor):
            raise ValueError('mascara errada')

        self._mascara = valor
        self._mascara_bin = self._ip_to_bin(valor)

        if not hasattr(self, 'prefixo'): #Impede a recursão prefixo-mascara
            self.prefixo = self._mascara_bin.count('1')

    @prefixo.setter
    def prefixo(self, valor):
        if not valor:
            return

        if not isinstance(valor, int): #Verifica se é inteiro
            raise TypeError('prefixo errado, tem q ser inteiro')

        if valor > 32:
            raise TypeError('prefixo errado, tem q ser menor que 32')

        self._prefixo = valor
        self._mascara_bin = (valor * '1').ljust(32, '0')
        if not hasattr(self, 'mascara'): #Impede a recursão prefixo-mascara
            self._bin_to_ip(self._mascara_bin)

    @staticmethod
    def _valida_ip(ip):
        regexp = re.compile( #Cria o compile que busca 4 blocos de 1-3 digitos de 0-9
            r'^([0-9]{1,3}).([0-9]{1,3}).([0-9]{1,3}).([0-9]{1,3})$'
        )
        if regexp.search(ip): 
            return True

    @staticmethod
    def _ip_to_bin(ip):
        '''Método que retorna o binário de um ip já validado'''
        blocos = ip.split('.')
        blocos_bin = [bin(int(x))[2:].zfill(8) for x in blocos]
        return ''.join(blocos_bin)

    @staticmethod
    def _bin_to_ip(ip):
        '''Método que retorna os numeros do binário'''
        n = 8
        blocos = [str(int(ip[i:n+i],2)) for i in range(0,32,n)]
        return '.'.join(blocos)

    def _set_broadcast(self):
        '''Método que descobre o broadcast a partir do prefixo e do ip'''
        host_bits = 32 - self.prefixo
        self._broadcast_bin = self._ip_bin[:self.prefixo] + (host_bits * '1')
        self._broadcast = self._bin_to_ip(self._broadcast_bin)
        return self._broadcast

    def _set_rede(self):
        '''Método que descobre a rede a partir do prefixo e do ip'''
        host_bits = 32 - self.prefixo
        self._rede_bin = self._ip_bin[:self.prefixo] + (host_bits * '0')
        self._rede = self._bin_to_ip(self._rede_bin)
        return self._rede

    def _get_numero_ips(self):
        '''Método que retorna 2 e^(32-prefixo) que '''
        return 2 ** (32 - self.prefixo)